# Imports

In [1]:
import logging
import os

import pandas as pd

import core.config.config_ as cconconf
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint
import helpers.hs3 as hs3
import im.cryptodatadownload.data.load.loader as imcdalolo
import im_v2.ccxt.data.client.clients as imvcdclcl
import im_v2.ccxt.universe.universe as imvccunun
import im_v2.common.data.client.clients as ivcdclcl
import research_amp.cc.statistics as ramccsta

In [2]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-a2193cbc-bb05-4b15-8a3e-40a0920b46aa.json'
>>ENV<<: is_inside_container=True: code_version=cmamp-1.0.3, container_version=cmamp-1.0.3, is_inside_docker=True, is_inside_ci=False, CI_defined=True, CI=''
>>ENV<<: AM_AWS_PROFILE=True AM_ECR_BASE_PATH=True AM_S3_BUCKET=True AM_TELEGRAM_TOKEN=True AWS_ACCESS_KEY_ID=False AWS_DEFAULT_REGION=False AWS_SECRET_ACCESS_KEY=False GH_ACTION_ACCESS_TOKEN=True
generated new fontManager
# Git
    branch_name='CMTask833_Unify_CDD_loader'
    hash='934922f40'
    # Last commits:
      * 934922f40 Maksim Sergeychikov CMTask833: Apply new changes                                      ( 4 minutes ago) Wed Dec 29 15:48:24 2021  (HEAD -> CMTask833_Unify_CDD_loader)
      * eef65b244 Maksim Sergeychikov CMTask833: Fix data_type                                          (   5 hours ago) Wed Dec 29 11:08:22 2021  (origin/CMTask833_Unify_CDD_loader)
   

# Configs

In [3]:
# Two configs are necessary in this situation because current downloading functions 
# work only with specific 'vendor' value.

In [4]:
def get_cmtask324_config_ccxt() -> cconconf.Config:
    """
    Get task232-specific config.
    """
    config = cconconf.Config()
    # Load parameters.
    config.add_subconfig("load")
    config["load"]["aws_profile"] = "am"
    config["load"]["data_dir"] = os.path.join(hs3.get_path(), "data")
    # Data parameters.
    config.add_subconfig("data")
    config["data"]["data_type"] = "OHLCV"
    config["data"]["target_frequency"] = "T"
    config["data"]["universe_version"] = "v03"
    config["data"]["vendor"] = "CCXT"
    # Column names.
    config.add_subconfig("column_names")
    config["column_names"]["close_price"] = "close"
    config["column_names"]["currency_pair"] = "currency_pair"
    config["column_names"]["exchange_id"] = "exchange_id"
    return config

In [5]:
config_ccxt = get_cmtask324_config_ccxt()
print(config_ccxt)

load:
  aws_profile: am
  data_dir: s3://alphamatic-data/data
data:
  data_type: OHLCV
  target_frequency: T
  universe_version: v03
  vendor: CCXT
column_names:
  close_price: close
  currency_pair: currency_pair
  exchange_id: exchange_id


In [6]:
def get_cmtask324_config_cdd() -> cconconf.Config:
    """
    Get task324-specific config.
    """
    config = cconconf.Config()
    # Load parameters.
    config.add_subconfig("load")
    config["load"]["aws_profile"] = "am"
    config["load"]["data_dir"] = os.path.join(hs3.get_path(), "data")
    # Data parameters.
    config.add_subconfig("data")
    config["data"]["data_type"] = "OHLCV"
    config["data"]["target_frequency"] = "T"
    config["data"]["universe_version"] = "v01"
    config["data"]["vendor"] = "CDD"
    # Column names.
    config.add_subconfig("column_names")
    config["column_names"]["close_price"] = "close"
    config["column_names"]["currency_pair"] = "currency_pair"
    config["column_names"]["exchange_id"] = "exchange_id"
    return config

In [7]:
config_cdd = get_cmtask324_config_cdd()
print(config_cdd)

load:
  aws_profile: am
  data_dir: s3://alphamatic-data/data
data:
  data_type: OHLCV
  target_frequency: T
  universe_version: v01
  vendor: CDD
column_names:
  close_price: close
  currency_pair: currency_pair
  exchange_id: exchange_id


# Load the data universe

## CCXT

In [8]:
ccxt_universe = imvccunun.get_vendor_universe(version="v03")

## CDD

In [9]:
cdd_universe = imvccunun.get_vendor_universe(version="v01", vendor="CDD")
# Remove non-USDT elements, since we are not interested in them.
cdd_universe = [element for element in cdd_universe if element.endswith("USDT")]

# Compare universes

In [10]:
_LOG.info("Number of full symbols in CCXT: %s", len(ccxt_universe))
_LOG.info("Number of full symbols in CDD: %s", len(cdd_universe))

Number of full symbols in CCXT: 38
Number of full symbols in CDD: 58


In [11]:
# Intersection of full symbols between two vendors.
currency_pair_intersection = set(ccxt_universe).intersection(cdd_universe)
_LOG.info("Number of similar full symbols: %s", len(currency_pair_intersection))
display(currency_pair_intersection)

Number of similar full symbols: 18


{'binance::ADA_USDT',
 'binance::BNB_USDT',
 'binance::BTC_USDT',
 'binance::EOS_USDT',
 'binance::ETH_USDT',
 'binance::LINK_USDT',
 'binance::SOL_USDT',
 'ftx::BNB_USDT',
 'ftx::BTC_USDT',
 'ftx::ETH_USDT',
 'ftx::LINK_USDT',
 'ftx::XRP_USDT',
 'kucoin::ADA_USDT',
 'kucoin::BNB_USDT',
 'kucoin::BTC_USDT',
 'kucoin::EOS_USDT',
 'kucoin::ETH_USDT',
 'kucoin::XRP_USDT'}

In [12]:
# Full symbols that are included in CCXT but not in CDD.
ccxt_and_not_cdd = set(ccxt_universe).difference(cdd_universe)
_LOG.info(
    "Number of full symbols that are included in CCXT but not in CDD: %s",
    len(ccxt_and_not_cdd),
)
display(ccxt_and_not_cdd)

Number of full symbols that are included in CCXT but not in CDD: 20


{'binance::AVAX_USDT',
 'binance::DOGE_USDT',
 'ftx::DOGE_USDT',
 'ftx::SOL_USDT',
 'gateio::ADA_USDT',
 'gateio::AVAX_USDT',
 'gateio::BNB_USDT',
 'gateio::BTC_USDT',
 'gateio::DOGE_USDT',
 'gateio::EOS_USDT',
 'gateio::ETH_USDT',
 'gateio::FIL_USDT',
 'gateio::LINK_USDT',
 'gateio::SOL_USDT',
 'gateio::XRP_USDT',
 'kucoin::AVAX_USDT',
 'kucoin::DOGE_USDT',
 'kucoin::FIL_USDT',
 'kucoin::LINK_USDT',
 'kucoin::SOL_USDT'}

In [13]:
# Full symbols that are included in CDD but not in CCXT.
cdd_and_not_ccxt = set(cdd_universe).difference(ccxt_universe)
_LOG.info(
    "Number of full symbols that are included in CDD but not in CCXT: %s",
    len(cdd_and_not_ccxt),
)
display(cdd_and_not_ccxt)

Number of full symbols that are included in CDD but not in CCXT: 40


{'binance::AAVE_USDT',
 'binance::BAT_USDT',
 'binance::BTT_USDT',
 'binance::CELR_USDT',
 'binance::CVC_USDT',
 'binance::DAI_USDT',
 'binance::DASH_USDT',
 'binance::DOT_USDT',
 'binance::ETC_USDT',
 'binance::FIL_USDT',
 'binance::ICP_USDT',
 'binance::ICX_USDT',
 'binance::LRC_USDT',
 'binance::LTC_USDT',
 'binance::MATIC_USDT',
 'binance::MKR_USDT',
 'binance::NEO_USDT',
 'binance::ONE_USDT',
 'binance::PAX_USDT',
 'binance::QTUM_USDT',
 'binance::SCU_USDT',
 'binance::TRX_USDT',
 'binance::TUSD_USDT',
 'binance::UNI_USDT',
 'binance::USDC_USDT',
 'binance::VET_USDT',
 'binance::XLM_USDT',
 'binance::XMR_USDT',
 'binance::XRP_USDT',
 'binance::ZEC_USDT',
 'ftx::BCH_USDT',
 'ftx::LTC_USDT',
 'ftx::TRX_USDT',
 'kucoin::BCH_USDT',
 'kucoin::DASH_USDT',
 'kucoin::LTC_USDT',
 'kucoin::NEO_USDT',
 'kucoin::TRX_USDT',
 'kucoin::XTZ_USDT',
 'kucoin::ZEC_USDT'}

# Compare close prices / returns from Binance

## Load the data

The code below can be used to load all the existing data from two vendors CDD and CCXT. Current version is specified to Binance only, however, even for one exchange there's too many data to operate, that's why the output is the intersection of currency pairs between to universe, since one can compare only the intersection of currency pairs for two vendors.

In [14]:
# Load Binance-specific universe for CCXT.
ccxt_binance_universe = [
    element for element in ccxt_universe if element.startswith("binance")
]
# Load Binnance-specific universe for CDD.
cdd_binance_universe_initial = [
    element for element in cdd_universe if element.startswith("binance")
]
cdd_binance_universe = cdd_binance_universe_initial.copy()
# SCU_USDT has incorrect columns, so can not be downloaded.
# See CMTask244 - Cannot load CDD - binance - SCU/USDT from s3 for the reference.
cdd_binance_universe.remove("binance::SCU_USDT")
# The intersection of Binance currency pairs from two universes.
currency_pair_intersection_binance = set(ccxt_binance_universe).intersection(
    cdd_binance_universe_initial
)

In [15]:
root_dir = os.path.join(hs3.get_path(), "data")

In [16]:
cdd_data = []
cdd_loader = imcdalolo.CddLoader(
    data_type="ohlcv", root_dir=root_dir, aws_profile="am"
)

for full_symbol in currency_pair_intersection_binance:
    cur_data = cdd_loader.read_data(full_symbol=full_symbol)
    cdd_data.append(cur_data)
cdd_binance_df = pd.concat(cdd_data)

Reading CDD data for exchange id='binance', currencies='SOL_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/SOL_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='SOL_USDT'...
Index length increased by 270 = 129252 - 128982
Reading CDD data for exchange id='binance', currencies='BTC_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/BTC_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='BTC_USDT'...
Index length increased by 5289 = 1063179 - 1057890
Reading CDD data for exchange id='binance', currencies='LINK_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/LINK_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='LINK_USDT'...
Index length increased by 5289 = 875139 - 869850
Reading CDD data for exchange id='binance', currencies='BNB_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/BNB_USDT.csv.

In [17]:
display(cdd_binance_df.head(3))
display(cdd_binance_df.shape)

,open,high,low,close,volume,epoch,currency_pair,exchange_id
2019-11-27 07:45:00+00:00,146.0,146.0,146.0,146.0,0.01,1.574841e+12,ETH/USDT,binance
2019-11-27 07:46:00+00:00,146.0,146.0,146.0,146.0,0.00,1.574841e+12,ETH/USDT,binance
2019-11-27 07:47:00+00:00,146.0,146.0,146.0,146.0,0.00,1.574841e+12,ETH/USDT,binance


(5599808, 8)

In [34]:
ccxt_client = imvcdclcl.CcxtCsvFileSystemClient(
    data_type="ohlcv", root_dir=root_dir, aws_profile="am"
)
multiple_symbols_client = ivcdclcl.MultipleSymbolsImClient(
    class_=ccxt_client, mode="concat"
)
ccxt_binance_df = multiple_symbols_client.read_data(
    currency_pair_intersection_binance
)

Reading CCXT data for exchange id='binance', currencies='ADA_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/ADA_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='ADA_USDT'...
Index length increased by 4520 = 1622000 - 1617480
Reading CCXT data for exchange id='binance', currencies='BNB_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/BNB_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='BNB_USDT'...
Index length increased by 4520 = 1622295 - 1617775
Reading CCXT data for exchange id='binance', currencies='BTC_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/BTC_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='BTC_USDT'...
Index length increased by 4519 = 1619641 - 1615122
Reading CCXT data for exchange id='binance', currencies='EOS_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/EOS_USDT.csv.gz'...
Processing CCXT data for exchange id='binanc

In [35]:
ccxt_binance_df = ccxt_binance_df.sort_index()

In [36]:
display(ccxt_binance_df.head(3))
display(ccxt_binance_df.shape)

,full_symbol,open,high,low,close,volume,epoch,currency_pair,exchange_id
2018-08-17 00:00:00+00:00,binance::ADA_USDT,0.0946,0.0948,0.09442,0.09479,41334.200000,1.534464e+12,ADA_USDT,binance
2018-08-17 00:00:00+00:00,binance::BTC_USDT,6316.0000,6319.0400,6310.32000,6311.64000,9.967395,1.534464e+12,BTC_USDT,binance
2018-08-17 00:00:00+00:00,binance::BNB_USDT,9.7779,9.7791,9.75380,9.77780,520.660000,1.534464e+12,BNB_USDT,binance


(10084929, 9)

## Calculate returns and correlation

In [37]:
# CDD names cleaning.
cdd_binance_df["currency_pair"] = cdd_binance_df["currency_pair"].str.replace(
    "/", "_"
)

In [38]:
def resample_close_price(df: pd.DataFrame, resampling_freq: str) -> pd.Series:
    """
    Resample close price on the currency level to the specified frequency using
    the last close price.

    :param df: OHLCV data
    :param resampling_freq: frequency from `pd.date_range()` to resample to
    :return: resampled close price per currency
    """
    # Reseting DateTime index, since pd.Grouper can't use index values.
    df = df.reset_index().rename(columns={"index": "stamp"})
    # Group by currency pairs and simultaneously resample to the desired frequency.
    resampler = df.groupby(
        ["currency_pair", pd.Grouper(key="stamp", freq=resampling_freq)]
    )
    # Take the last close value from each resampling period.
    close_series = resampler.close.last()
    return close_series

In [39]:
def calculate_correlations(
    ccxt_close_price: pd.Series, cdd_close_price: pd.Series, compute_returns: bool
) -> pd.DataFrame:
    """
    Take CCXT and CDD close prices and calculate the correlations for each
    specific currency pair.

    :param ccxt_series: resampled close price per currency for CCXT
    :param cdd_series: resampled close price per currency for CDD
    :param compute_returns: if True - compare returns, if False - compare close prices
    :return: correlation matrix per currency
    """
    if compute_returns:
        # Group by currency pairs in order to calculate the percentage returns.
        grouper_cdd = cdd_close_price.groupby("currency_pair")
        cdd_close_price = grouper_cdd.pct_change()
        grouper_ccxt = ccxt_close_price.groupby("currency_pair")
        ccxt_close_price = grouper_ccxt.pct_change()
    # Combine and calculate correlations.
    combined = pd.merge(
        cdd_close_price, ccxt_close_price, left_index=True, right_index=True
    )
    # Rename the columns.
    if compute_returns:
        combined.columns = ["ccxt_returns", "cdd_returns"]
    else:
        combined.columns = ["cdd_close", "ccxt_close"]
    # Group by again to calculte returns correlation for each currency pair.
    corr_matrix = combined.groupby(level=0).corr()
    return corr_matrix

In [40]:
# Corresponding resampled Series.
ccxt_binance_series_1d = resample_close_price(ccxt_binance_df, "1D")
cdd_binance_series_1d = resample_close_price(cdd_binance_df, "1D")

ccxt_binance_series_5min = resample_close_price(ccxt_binance_df, "5min")
cdd_binance_series_5min = resample_close_price(cdd_binance_df, "5min")

### 1-day returns

In [20]:
returns_corr_1day = calculate_correlations(
    ccxt_binance_series_1d, cdd_binance_series_1d, compute_returns=True
)
display(returns_corr_1day)

ccxt_returns  cdd_returns
currency_pair                                        
ADA_USDT      ccxt_returns      1.000000     0.997807
              cdd_returns       0.997807     1.000000
BNB_USDT      ccxt_returns      1.000000     0.998490
              cdd_returns       0.998490     1.000000
BTC_USDT      ccxt_returns      1.000000     0.997763
              cdd_returns       0.997763     1.000000
EOS_USDT      ccxt_returns      1.000000     0.998294
              cdd_returns       0.998294     1.000000
ETH_USDT      ccxt_returns      1.000000     0.995563
              cdd_returns       0.995563     1.000000
LINK_USDT     ccxt_returns      1.000000     0.997200
              cdd_returns       0.997200     1.000000
SOL_USDT      ccxt_returns      1.000000     0.996158
              cdd_returns       0.996158     1.000000

### 5-min returns

In [21]:
returns_corr_5min = calculate_correlations(
    ccxt_binance_series_5min, cdd_binance_series_5min, compute_returns=True
)
display(returns_corr_5min)

ccxt_returns  cdd_returns
currency_pair                                        
ADA_USDT      ccxt_returns      1.000000     0.986912
              cdd_returns       0.986912     1.000000
BNB_USDT      ccxt_returns      1.000000     0.987292
              cdd_returns       0.987292     1.000000
BTC_USDT      ccxt_returns      1.000000     0.990746
              cdd_returns       0.990746     1.000000
EOS_USDT      ccxt_returns      1.000000     0.993959
              cdd_returns       0.993959     1.000000
ETH_USDT      ccxt_returns      1.000000     0.974821
              cdd_returns       0.974821     1.000000
LINK_USDT     ccxt_returns      1.000000     0.983939
              cdd_returns       0.983939     1.000000
SOL_USDT      ccxt_returns      1.000000     0.999998
              cdd_returns       0.999998     1.000000

## Compare close prices

### 1-day close prices

In [22]:
close_corr_1day = calculate_correlations(
    ccxt_binance_series_1d, cdd_binance_series_1d, compute_returns=False
)
display(close_corr_1day)

cdd_close  ccxt_close
currency_pair                                  
ADA_USDT      cdd_close    1.000000    0.999995
              ccxt_close   0.999995    1.000000
BNB_USDT      cdd_close    1.000000    0.999999
              ccxt_close   0.999999    1.000000
BTC_USDT      cdd_close    1.000000    1.000000
              ccxt_close   1.000000    1.000000
EOS_USDT      cdd_close    1.000000    0.999906
              ccxt_close   0.999906    1.000000
ETH_USDT      cdd_close    1.000000    0.999994
              ccxt_close   0.999994    1.000000
LINK_USDT     cdd_close    1.000000    0.999995
              ccxt_close   0.999995    1.000000
SOL_USDT      cdd_close    1.000000    0.999474
              ccxt_close   0.999474    1.000000

### 5-min close prices

In [41]:
close_corr_5min = calculate_correlations(
    ccxt_binance_series_5min, cdd_binance_series_5min, compute_returns=False
)
display(close_corr_5min)

cdd_close  ccxt_close
currency_pair                                  
ADA_USDT      cdd_close    1.000000    1.000000
              ccxt_close   1.000000    1.000000
BNB_USDT      cdd_close    1.000000    1.000000
              ccxt_close   1.000000    1.000000
BTC_USDT      cdd_close    1.000000    1.000000
              ccxt_close   1.000000    1.000000
EOS_USDT      cdd_close    1.000000    0.999999
              ccxt_close   0.999999    1.000000
ETH_USDT      cdd_close    1.000000    1.000000
              ccxt_close   1.000000    1.000000
LINK_USDT     cdd_close    1.000000    1.000000
              ccxt_close   1.000000    1.000000
SOL_USDT      cdd_close    1.000000    1.000000
              ccxt_close   1.000000    1.000000

# Statistical properties of a full symbol in CDD

In [14]:
# Clearing CDD currency pairs that are incorrect.

# Binance
cdd_universe.remove("binance::SCU_USDT")

# ftx has some critical mistakes in the downloading process, so can not continue analysis with them.
# see CMTask801 - Downloading issues of FTX exchange from CDD universe for further reference.
cdd_ftx_universe = [
    element for element in cdd_universe if element.startswith("ftx")
]
for elem in cdd_ftx_universe:
    cdd_universe.remove(elem)

# kucoin exchange: the timestamps are obviously wrong and with too short time period.
# see CMTask253 - Fix timestamp for CDD - kucoin for reference.
cdd_kucoin_universe = [
    element for element in cdd_universe if element.startswith("kucoin")
]
for elem in cdd_kucoin_universe:
    cdd_universe.remove(elem)

## Comparison of intersection of full symbols between CCXT and CDD

In [15]:
# Full symbols that are included in CDD but not in CCXT (cleaned from unavailable full symbols).
cdd_and_ccxt_cleaned = set(ccxt_universe).intersection(cdd_universe)
len(cdd_and_ccxt_cleaned)

7

### Load the intersection of full symbols for CDD and CCXT

#### CDD

In [16]:
compute_start_end_stats = lambda data: ramccsta.compute_start_end_stats(
    data, config_cdd
)

cdd_start_end_table = ramccsta.compute_stats_for_universe(
    cdd_and_ccxt_cleaned, config_cdd, compute_start_end_stats
)

Reading CDD data for exchange id='binance', currencies='BNB_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/BNB_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='BNB_USDT'...
Index length increased by 5289 = 840575 - 835286
Reading CDD data for exchange id='binance', currencies='EOS_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/EOS_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='EOS_USDT'...
Index length increased by 5289 = 888096 - 882807
Reading CDD data for exchange id='binance', currencies='SOL_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/SOL_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='SOL_USDT'...
Index length increased by 270 = 129252 - 128982
Reading CDD data for exchange id='binance', currencies='BTC_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/BTC_USDT.csv.gz'..

In [17]:
# CDD names cleaning.
cdd_start_end_table["currency_pair"] = cdd_start_end_table[
    "currency_pair"
].str.replace("/", "_")

In [18]:
cdd_start_end_table.head(3)

,exchange_id,currency_pair,min_timestamp,max_timestamp,n_data_points,coverage,days_available,avg_data_points_per_day,longest_not_nan_seq_days,longest_not_nan_seq_perc,longest_not_nan_seq_start_date,longest_not_nan_seq_end_date,vendor
0,binance,BNB_USDT,2020-02-10 08:01:00+00:00,2021-09-16 01:35:00+00:00,835286,99.370788,583,1432.737564,293,50.351129,2020-02-10 08:01:00+00:00,2020-11-30 05:59:00+00:00,CDD
1,binance,EOS_USDT,2020-01-08 08:00:00+00:00,2021-09-16 01:35:00+00:00,882807,99.404456,616,1433.128247,326,53.007783,2020-01-08 08:00:00+00:00,2020-11-30 05:59:00+00:00,CDD
2,binance,SOL_USDT,2021-06-06 06:57:00+00:00,2021-09-04 01:08:00+00:00,128982,99.791106,89,1449.235955,67,75.529199,2021-06-06 06:57:00+00:00,2021-08-13 01:59:00+00:00,CDD


#### CCXT

In [19]:
compute_start_end_stats = lambda data: ramccsta.compute_start_end_stats(
    data, config_ccxt
)

ccxt_start_end_table = ramccsta.compute_stats_for_universe(
    cdd_and_ccxt_cleaned, config_ccxt, compute_start_end_stats
)

Reading CCXT data for exchange id='binance', currencies='BNB_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/BNB_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='BNB_USDT'...
Index length increased by 4520 = 1622295 - 1617775
Reading CCXT data for exchange id='binance', currencies='EOS_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/EOS_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='EOS_USDT'...
Index length increased by 4520 = 1622444 - 1617924
Reading CCXT data for exchange id='binance', currencies='SOL_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/SOL_USDT.csv.gz'...
Processing CCXT data for exchange id='binance', currencies='SOL_USDT'...
Index length increased by 1224 = 576140 - 574916
Reading CCXT data for exchange id='binance', currencies='BTC_USDT' from file='s3://alphamatic-data/data/ccxt/20210924/binance/BTC_USDT.csv.gz'...
Processing CCXT data for exchange id='binance'

In [20]:
ccxt_start_end_table.head(3)

,exchange_id,currency_pair,min_timestamp,max_timestamp,n_data_points,coverage,days_available,avg_data_points_per_day,longest_not_nan_seq_days,longest_not_nan_seq_perc,longest_not_nan_seq_start_date,longest_not_nan_seq_end_date,vendor
0,binance,BNB_USDT,2018-08-17 00:00:00+00:00,2021-09-16 14:14:00+00:00,1617775,99.721382,1126,1436.745115,155,13.760136,2020-06-28 05:30:00+00:00,2020-11-30 05:59:00+00:00,CCXT
1,binance,EOS_USDT,2018-08-17 00:00:00+00:00,2021-09-16 16:43:00+00:00,1617924,99.721408,1126,1436.877442,155,13.758872,2020-06-28 05:30:00+00:00,2020-11-30 05:59:00+00:00,CCXT
2,binance,SOL_USDT,2020-08-11 06:00:00+00:00,2021-09-15 08:19:00+00:00,574916,99.787552,400,1437.29,110,27.743257,2020-08-11 06:00:00+00:00,2020-11-30 05:59:00+00:00,CCXT


### Display the union results

In [23]:
def unify_start_end_tables(
    cdd_df: pd.DataFrame, ccxt_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Combine CCXT and CDD start-end stats tables into unique table.

    :param cdd_df: start-end table for CCXT
    :param ccxt_df: start-end table for CDD
    :return: unified start-end table
    """
    # set Multiindex.
    cdd_df = cdd_df.set_index(["exchange_id", "currency_pair"])
    ccxt_df = ccxt_df.set_index(["exchange_id", "currency_pair"])
    # add suffixes.
    ccxt_df = ccxt_df.add_suffix("_ccxt")
    cdd_df = cdd_df.add_suffix("_cdd")
    # combine two universes.
    ccxt_and_cdd = pd.concat([cdd_df, ccxt_df], axis=1)
    # sort columns.
    cols_to_sort = ccxt_and_cdd.columns.to_list()
    ccxt_and_cdd = ccxt_and_cdd[sorted(cols_to_sort)]
    return ccxt_and_cdd

In [24]:
union_cdd_ccxt_stats = unify_start_end_tables(
    cdd_start_end_table, ccxt_start_end_table
)
display(union_cdd_ccxt_stats)

avg_data_points_per_day_ccxt avg_data_points_per_day_cdd coverage_ccxt coverage_cdd days_available_ccxt days_available_cdd longest_not_nan_seq_days_ccxt longest_not_nan_seq_days_cdd longest_not_nan_seq_end_date_ccxt longest_not_nan_seq_end_date_cdd longest_not_nan_seq_perc_ccxt longest_not_nan_seq_perc_cdd longest_not_nan_seq_start_date_ccxt longest_not_nan_seq_start_date_cdd        max_timestamp_ccxt         max_timestamp_cdd        min_timestamp_ccxt         min_timestamp_cdd n_data_points_ccxt n_data_points_cdd vendor_ccxt vendor_cdd
exchange_id currency_pair                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
binance     BNB_USDT                       1436.745115                 1432.737564     99.721382    99.370788                1126                583                           155                          293         2020-11-30 05:59:00+00:00        2020-11-30 05:59:00+00:00                     13.760136                    50.351129           2020-06-28 05:30:00+00:00          2020-02-10 08:01:00+00:00 2021-09-16 14:14:00+00:00 2021-09-16 01:35:00+00:00 2018-08-17 00:00:00+00:00 2020-02-10 08:01:00+00:00            1617775            835286        CCXT        CDD
            EOS_USDT                       1436.877442                 1433.128247     99.721408    99.404456                1126                616                           155                          326         2020-11-30 05:59:00+00:00        2020-11-30 05:59:00+00:00                     13.758872                    53.007783           2020-06-28 05:30:00+00:00          2020-01-08 08:00:00+00:00 2021-09-16 16:43:00+00:00 2021-09-16 01:35:00+00:00 2018-08-17 00:00:00+00:00 2020-01-08 08:00:00+00:00            1617924            882807        CCXT        CDD
            SOL_USDT                           1437.29                 1449.235955     99.787552    99.791106                 400                 89                           110                           67         2020-11-30 05:59:00+00:00        2021-08-13 01:59:00+00:00                     27.743257                    75.529199           2020-08-11 06:00:00+00:00          2021-06-06 06:57:00+00:00 2021-09-15 08:19:00+00:00 2021-09-04 01:08:00+00:00 2020-08-11 06:00:00+00:00 2021-06-06 06:57:00+00:00             574916            128982        CCXT        CDD
            BTC_USDT                       1436.941281                 1433.455285     99.720988     99.50253                1124                738                           155                          448         2020-11-30 05:59:00+00:00        2020-11-30 05:59:00+00:00                     13.782684                    60.740383           2020-06-28 05:30:00+00:00          2019-09-08 19:01:00+00:00 2021-09-14 18:00:00+00:00 2021-09-16 01:35:00+00:00 2018-08-17 00:00:00+00:00 2019-09-08 17:57:00+00:00            1615122           1057890        CCXT        CDD
            LINK_USDT                      1437.440904                 1433.031301     99.722642    99.395639                 973                607                           155                          317         2020-11-30 05:59:00+00:00        2020-11-30 05:59:00+00:00                     15.916351                     52.31169           2020-06-28 05:30:00+00:00          2020-01-17 08:00:00+00:00 2021-09-16 09:19:00+00:00 2021-09-16 01:38:00+00:00 2019-01-16 10:00:00+00:00 2020-01-17 08:00:00+00:00            1398630            869850        CCXT        CDD
            ADA_USDT                       

## Comparison of full symbols that are included in CDD but not available in CCXT

In [147]:
# Set of full symbols that are included in CDD but not available in CCXT (cleaned from unavailable full symbols).
cdd_and_not_ccxt_cleaned = set(cdd_universe).difference(ccxt_universe)
len(cdd_and_not_ccxt_cleaned)

29

In [150]:
# for 'avg_data_points_per_day' the amount of "days_available" is equal to 0, so it crashes the calculations.
cdd_and_not_ccxt_cleaned.remove("binance::DAI_USDT")

In [154]:
compute_start_end_stats = lambda data: ramccsta.compute_start_end_stats(
    data, config_cdd
)

cdd_unique_start_end_table = ramccsta.compute_stats_for_universe(
    cdd_and_not_ccxt_cleaned, config_cdd, compute_start_end_stats
)

Reading CDD data for exchange id='binance', currencies='NEO_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/NEO_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='NEO_USDT'...
Index length increased by 5289 = 830452 - 825163
Reading CDD data for exchange id='binance', currencies='LRC_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/LRC_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='LRC_USDT'...
Index length increased by 270 = 444576 - 444306
Reading CDD data for exchange id='binance', currencies='USDC_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/USDC_USDT.csv.gz'...
Processing CDD data for exchange id='binance', currencies='USDC_USDT'...
Index length increased by 5288 = 531656 - 526368
Reading CDD data for exchange id='binance', currencies='VET_USDT', from file='s3://alphamatic-data/data/cryptodatadownload/20210924/binance/VET_USDT.csv.gz

In [156]:
display(cdd_unique_start_end_table)

,exchange_id,currency_pair,min_timestamp,max_timestamp,n_data_points,coverage,days_available,avg_data_points_per_day,longest_not_nan_seq_days,longest_not_nan_seq_perc,longest_not_nan_seq_start_date,longest_not_nan_seq_end_date,vendor
0,binance,NEO/USDT,2020-02-17 08:45:00+00:00,2021-09-16 01:36:00+00:00,825163,99.363118,576,1432.574653,286,49.745801,2020-02-17 08:45:00+00:00,2020-11-30 05:59:00+00:00,CDD
1,binance,LRC/USDT,2020-10-19 07:05:00+00:00,2021-08-24 00:40:00+00:00,444306,99.939268,308,1442.551948,297,96.454824,2020-10-19 07:05:00+00:00,2021-08-13 01:59:00+00:00,CDD
2,binance,USDC/USDT,2020-09-11 20:41:00+00:00,2021-09-16 01:36:00+00:00,526368,99.005372,369,1426.471545,79,21.502438,2020-09-11 20:41:00+00:00,2020-11-30 05:59:00+00:00,CDD
3,binance,VET/USDT,2021-06-06 06:57:00+00:00,2021-09-04 01:08:00+00:00,128982,99.791106,89,1449.235955,67,75.529199,2021-06-06 06:57:00+00:00,2021-08-13 01:59:00+00:00,CDD
4,binance,FIL/USDT,2021-06-06 06:57:00+00:00,2021-09-04 01:08:00+00:00,128982,99.791106,89,1449.235955,67,75.529199,2021-06-06 06:57:00+00:00,2021-08-13 01:59:00+00:00,CDD
5,binance,CELR/USDT,2020-09-11 20:41:00+00:00,2021-09-16 01:36:00+00:00,526368,99.005372,369,1426.471545,79,21.502438,2020-09-11 20:41:00+00:00,2020-11-30 05:59:00+00:00,CDD
6,binance,DASH/USDT,2020-02-04 08:06:00+00:00,2021-09-16 01:36:00+00:00,843922,99.377187,589,1432.804754,299,50.855912,2020-02-04 08:06:00+00:00,2020-11-30 05:59:00+00:00,CDD
7,binance,ONE/USDT,2020-09-11 20:41:00+00:00,2021-09-16 01:36:00+00:00,526368,99.005372,369,1426.471545,79,21.502438,2020-09-11 20:41:00+00:00,2020-11-30 05:59:00+00:00,CDD
8,binance,MATIC/USDT,2020-09-11 20:41:00+00:00,2021-09-16 01:36:00+00:00,526368,99.005372,369,1426.471545,79,21.502438,2020-09-11 20:41:00+00:00,2020-11-30 05:59:00+00:00,CDD
9,binance,QTUM/USDT,2020-02-20 08:06:00+00:00,2021-09-16 01:36:00+00:00,820882,99.359818,573,1432.603839,283,49.485397,2020-02-20 08:06:00+00:00,2020-11-30 05:59:00+00:00,CDD
